# TME sur la classification de lettres manuscrites
## Format des données
Nous travaillerons sur des lettres manuscrites.
Les données sont fournies au format pickle (le standard de sérialisation python, particulièrement convivial). Pour les charger : 

In [1]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

with open('ressources/lettres.pkl', 'rb') as f:
    data = pkl.load(f, encoding='latin1') 
X = np.array(data.get('letters')) # récupération des données sur les lettres
Y = np.array(data.get('labels')) # récupération des étiquettes associées 

Les données sont dans un format original: une lettre est en fait une série d'angles (exprimés en degrés). Un exemple: 

In [51]:
states_num = X[0].size  # On stocke le nombre des états de chaque signal pour l'utiliser plus tard
X[0]

array([ 36.214493, 347.719116, 322.088898, 312.230957, 314.851013,
       315.487213, 313.556702, 326.534973, 141.288971, 167.606689,
       199.321594, 217.911087, 226.443298, 235.002472, 252.354492,
       270.045654, 291.665161, 350.934723,  17.892815,  20.281025,
        28.207161,  43.883423,  53.459026])

Lors de l'acquisition, un stylo intelligent a pris des mesures régulièrement dans le temps: chaque période correspond à un segment de droite et le stylo a calculé l'angle entre deux segments consécutifs... C'est l'information qui vous est fournie.

Pour afficher une lettre, il faut reconstruire la trajectoire enregistrée... C'est ce que fait la méthode ci-dessous: 

In [3]:
# affichage d'une lettre
def tracerLettre(let):
    a = -let*np.pi/180; # conversion en rad
    coord = np.array([[0, 0]]); # point initial
    for i in range(len(a)):
        x = np.array([[1, 0]]);
        rot = np.array([[np.cos(a[i]), -np.sin(a[i])],[ np.sin(a[i]),np.cos(a[i])]])
        xr = x.dot(rot) # application de la rotation
        coord = np.vstack((coord,xr+coord[-1,:]))
    plt.figure()
    plt.plot(coord[:,0],coord[:,1])
    plt.savefig("exlettre.png")
    return

In [58]:
tracerLettre(X[0])

##  Apprentissage d'un modèle CM (max de vraisemblance)
### 1. Discrétisation

**1 état = 1 angle**

Il est nécessaire de regrouper les angles en un nombre fini d'états (par exemple 20)
- définir un `intervalle = 360 / n_etats`
- discrétiser tous les signaux à l'aide de la formule `np.floor(x / intervalle)`

Donner le code de la méthode `discretise(x, d)` qui prend la base des signaux et retourne une base de signaux discrétisés.

In [72]:
# Afin d'implementer cette fonction demandée on n'utilise que les expressions de NumPy
def discretise(x, d):
    intervalle = 360 / d
    return np.array([np.array(np.floor(xi), dtype=int) for xi in x / intervalle])

# En décommentant ligne du code ci-dessous, vous pouvez vérifier que cette fonction marche
# aussi dans le cas où on a pour x le tableau des signaux
# print(discretise(X, 3))
discretise(X[0], 3)

[array([0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0,
       0])
 array([2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0])
 array([2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0,
       0])
 array([2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0])
 array([0, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0])
 array([2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0])
 array([2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0])
 array([0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0,
       0, 0])
 array([2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0])
 array([2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0])
 array([0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0,
       0, 0, 0])
 array([2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1])
 array([2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2,

array([0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0,
       0])

**VALIDATION :** code du premier signal avec une discrétisation sur 3 états:
```python
array([ 0.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  1.,  1.,  1., 1., 1., 2., 2.,  2.,
       2.,  0.,  0.,  0.,  0.,  0.])
```

### 2. Regrouper les indices des signaux par classe (pour faciliter l'apprentissage)

In [6]:
def groupByLabel(y):
    index = []
    for i in np.unique(y): # pour toutes les classes
        ind, = np.where(y == i)
        index.append(ind)
    return index

groupByLabel(Y)

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]),
 array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),
 array([33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]),
 array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]),
 array([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]),
 array([66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]),
 array([77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]),
 array([88, 89, 90, 91, 92, 93, 94, 95, 96, 97]),
 array([ 98,  99, 100, 101, 102, 103, 104, 105, 106, 107]),
 array([108, 109, 110, 111, 112, 113, 114, 115, 116, 117]),
 array([118, 119, 120, 121, 122, 123, 124, 125, 126, 127]),
 array([128, 129, 130, 131, 132, 133, 134, 135, 136, 137]),
 array([138, 139, 140, 141, 142, 143, 144, 145, 146, 147]),
 array([148, 149, 150, 151, 152, 153, 154, 155, 156, 157]),
 array([158, 159, 160, 161, 162, 163, 164, 165, 166, 167]),
 array([168, 169, 170, 171, 172, 173, 174, 175, 176, 177]),
 array([178, 179, 180, 181, 182, 183, 

Cette méthode produit simplement une structure type:
```python
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]),
 array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),
 array([33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]),
 array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]),
 array([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]),
 ...
```
Chaque ligne regroupe les indices de signaux correspondant à une classe 

### 3. Apprendre les modèles CM

Soit {$X_C$} la base de signaux discrétisés correspondant à une classe {$C$} et {$d$} le nombre d'états. Donner le code de la fonction `learnMarkovModel(Xc, d)` qui retourne un tuple contenant Pi et A.

Rappel:
- Initialisation de 
```python
 A = np.zeros((d, d))
 Pi = np.zeros(d)```
- Parcours de tous les signaux et incréments de A et Pi
- Normalisation (un peu réfléchie pour éviter les divisions par 0)
```python
A = A / np.maximum(A.sum(1).reshape(d, 1), 1) # normalisation
Pi = Pi / Pi.sum()```

**Note** : la solution proposée pour gérer le cas des lignes entièrement à 0 est naïve et n'est pas totalement satisfaisante. Comprendre pourquoi. On proposera une solution améliorée plus loin dans le TME. 

In [73]:
# On estime les paramètres Pi et A
# Pi : pour chaque signal on compte combien de fois signal est commencé par quelque état
# A : pour chaque signal on compte combien de fois quelque état fait une transition à
# quelque autre état
def learnMarkovModel(Xc, d):
    Pi, A = np.zeros(d), np.zeros((d, d))
    for Xc_i in Xc:
        Pi[Xc_i[0]] += 1                                    # comptage pour Pi
        for state_index in range(len(Xc_i)-1):
            A[Xc_i[state_index], Xc_i[state_index+1]] += 1  # comptage pour A
    
    # normalisation
    Pi = Pi / Pi.sum() if Pi.sum() != 0 else Pi
    A = A / np.maximum(A.sum(1).reshape(d, 1), 1)
    return Pi, A

learnMarkovModel(discretise(X[groupByLabel(Y)[0]], 3), 3)

(array([0.36363636, 0.        , 0.63636364]),
 array([[0.84444444, 0.06666667, 0.08888889],
        [0.        , 0.83333333, 0.16666667],
        [0.11382114, 0.06504065, 0.82113821]]))

**Validation :** premier modèle avec une discrétisation sur 3 états :
```python
(array([ 0.36363636,  0.        ,  0.63636364]),
 array([[ 0.84444444,  0.06666667,  0.08888889],
       [ 0.        ,  0.83333333,  0.16666667],
       [ 0.11382114,  0.06504065,  0.82113821]]))
```

### 4. Stocker les modèles dans une liste

Pour un usage ultérieur plus facile, on utilise le code suivant :

In [74]:
# On crée des modèles pour d = 3
d = 3                   # paramètre de discrétisation
Xd = discretise(X, d)    # application de la discrétisation
index = groupByLabel(Y)  # groupement des signaux par classe
models = []
for cl in range(len(np.unique(Y))): # parcours de toutes les classes et optimisation des modèles
    models.append(learnMarkovModel(Xd[index[cl]], d))

##  Test (affectation dans les classes sur critère MV)
### 1. (log)Probabilité d'une séquence dans un modèle

Donner le code de la méthode `probaSequence(s,Pi,A)` qui retourne la log-probabilité d'une séquence `s` dans le modèle {$\lambda=\{Pi,A\}$} 

In [75]:
# On calcule log de la proba demandée en utilisant une formule : 
# P(S = (s_1, s_2, ..., s_T)) = P(s_1) * P(s_2 | s_1)* ... * P(s_T | s_{T-1})
# où la proba première est donnée par Pi et toutes les autres sont données par A, en fait
def probaSequence(s, Pi, A):
    prob = Pi[s[0]]
    for state_ind in range(1, len(s)):
        prob *= A[s[state_ind-1], s[state_ind]]
        
    return np.log(prob)

res = []
# on calcule log de proba pour tous les modèles et on les affiche
for model in models:
    Pi, A = model
    res.append(probaSequence(discretise(X[0], 3), Pi, A))
np.array(res)

/Users/ivankachaikin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


array([-13.491086  ,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf, -12.48285678])

**VALIDATION :** probabilité du premier signal dans les 26 modèles avec une discrétisation sur 3 états :
```python
array([-13.491086  ,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf,         -inf,         -inf,         -inf,
               -inf, -12.48285678])
```

- Ce signal est-il bien classé ?
- D'où viennent tous les `-inf` ? 

### 2. Application de la méthode précédente pour tous les signaux et tous les modèles de lettres

L'application se fait en une ligne de code si vous avez respecté les spécifications précédentes : 

In [76]:
# On recrée des modèles pour d = 20
d = 20                   # paramètre de discrétisation
Xd = discretise(X, d)    # application de la discrétisation
index = groupByLabel(Y)  # groupement des signaux par classe
models = []
for cl in range(len(np.unique(Y))): # parcours de toutes les classes et optimisation des modèles
    models.append(learnMarkovModel(Xd[index[cl]], d))

proba = np.array([[probaSequence(Xd[i], models[cl][0], models[cl][1]) for i in range(len(Xd))]
                  for cl in range(len(np.unique(Y)))])

/Users/ivankachaikin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


### 3. Evaluation des performances

Pour l'évaluation, nous proposons l'approche suivante: 

In [77]:
# calcul d'une version numérique des Y :
Ynum = np.zeros(Y.shape)
for num, char in enumerate(np.unique(Y)):
    Ynum[Y == char] = num
    
# print(Y)
# print(Ynum)
    
# Calcul de la classe la plus probable :
pred = proba.argmax(0) # max colonne par colonne
# print(pred)

# Calcul d'un pourcentage de bonne classification :
np.where(pred != Ynum, 0.,1.).mean()

0.914179104477612

**INDICE DE PERFORMANCE :** 91% de bonne classification avec 20 états, 69% avec 3 états

## Biais d'évaluation, notion de sur-apprentissage

Dans le protocole précédent, nous avons triché:
- les données servent d'abord à apprendre les modèles...
- puis nous nous servons des mêmes données pour tester les modèles ! Les performances sont forcément bonnes ! 

Afin de palier le problème, nous allons diviser en deux la base de données: une partie servira à l'apprentissage des modèles, l'autre à leur évaluation. Pour effectuer la division, nous fournissons le code suivant: 

In [111]:
# separation app/test, pc=ratio de points en apprentissage
def separeTrainTest(y, pc):
    indTrain = []
    indTest = []
    for i in np.unique(y): # pour toutes les classes
        ind, = np.where(y == i)
        n = len(ind)
        indTrain.append(ind[np.random.permutation(n)][:int(np.floor(pc * n))])
        indTest.append(np.setdiff1d(ind, indTrain[-1]))
    return indTrain, indTest

# exemple d'utilisation
itrain, itest = separeTrainTest(Y, 0.8)

dans `itrain`, nous obtenons les indices des signaux qui doivent servir en apprentissage pour chaque classe :

In [112]:
itrain

[array([ 6,  7,  9, 10,  5,  2,  1,  0]),
 array([11, 13, 20, 21, 18, 19, 17, 14]),
 array([27, 23, 30, 22, 32, 25, 28, 29]),
 array([37, 36, 42, 43, 35, 38, 39, 34]),
 array([50, 52, 48, 51, 46, 49, 45, 53]),
 array([57, 60, 65, 61, 58, 64, 56, 55]),
 array([73, 71, 72, 74, 75, 70, 69, 66]),
 array([81, 77, 84, 80, 83, 87, 82, 78]),
 array([93, 92, 91, 95, 97, 89, 90, 94]),
 array([107, 106, 101, 103, 104, 100,  98, 102]),
 array([110, 109, 112, 113, 111, 108, 116, 115]),
 array([122, 124, 127, 123, 126, 119, 118, 125]),
 array([132, 133, 136, 129, 137, 131, 135, 134]),
 array([142, 144, 139, 143, 138, 141, 145, 140]),
 array([157, 148, 153, 152, 149, 150, 154, 155]),
 array([163, 158, 164, 165, 159, 160, 167, 162]),
 array([169, 170, 176, 175, 177, 168, 172, 171]),
 array([186, 182, 187, 180, 184, 179, 178, 185]),
 array([190, 195, 194, 197, 192, 189, 188, 196]),
 array([200, 205, 202, 203, 207, 198, 206, 199]),
 array([214, 208, 209, 216, 210, 213, 211, 217]),
 array([224, 219, 227,

**Note :** pour faciliter l'évaluation des modèles, vous aurez besoin de re-fusionner tous les indices d'apprentissage et de test. Cela se fait avec les lignes de code suivantes : 

In [113]:
ia = []
for i in itrain:
    ia += i.tolist()    
it = []
for i in itest:
    it += i.tolist()
print(ia)
print(it)

[6, 7, 9, 10, 5, 2, 1, 0, 11, 13, 20, 21, 18, 19, 17, 14, 27, 23, 30, 22, 32, 25, 28, 29, 37, 36, 42, 43, 35, 38, 39, 34, 50, 52, 48, 51, 46, 49, 45, 53, 57, 60, 65, 61, 58, 64, 56, 55, 73, 71, 72, 74, 75, 70, 69, 66, 81, 77, 84, 80, 83, 87, 82, 78, 93, 92, 91, 95, 97, 89, 90, 94, 107, 106, 101, 103, 104, 100, 98, 102, 110, 109, 112, 113, 111, 108, 116, 115, 122, 124, 127, 123, 126, 119, 118, 125, 132, 133, 136, 129, 137, 131, 135, 134, 142, 144, 139, 143, 138, 141, 145, 140, 157, 148, 153, 152, 149, 150, 154, 155, 163, 158, 164, 165, 159, 160, 167, 162, 169, 170, 176, 175, 177, 168, 172, 171, 186, 182, 187, 180, 184, 179, 178, 185, 190, 195, 194, 197, 192, 189, 188, 196, 200, 205, 202, 203, 207, 198, 206, 199, 214, 208, 209, 216, 210, 213, 211, 217, 224, 219, 227, 226, 225, 223, 221, 222, 234, 229, 228, 230, 236, 235, 232, 233, 246, 245, 247, 238, 240, 242, 239, 244, 256, 250, 249, 254, 255, 253, 257, 248, 267, 259, 264, 265, 260, 266, 263, 261]
[3, 4, 8, 12, 15, 16, 24, 26, 31, 33, 4

**Note 2 :** Du fait de la permutation aléatoire, les résultats vont bouger (un peu) à chaque execution du programme. 

## Questions importantes
- Ré-utiliser les fonctions précédemment définies pour apprendre des modèles et les évaluer sans biais.
- Calculer et analyser les résultats obtenus en apprentissage et en test
- Etudier l'évolution des performances en fonction de la discrétisation

In [120]:
# on crée des modèles à partir de l'échantillon de l'apprentissage en utilisant les index 
# obtenus ci-dessus
d = 5                              # paramètre de discrétisation
Xd_train = discretise(X[ia], d)    # application de la discrétisation
Xd_test = discretise(X[it], d)
index = itrain.copy()
models = []
for cl in range(len(np.unique(Y))): # parcours de toutes les classes et optimisation des modèles
    models.append(learnMarkovModel(discretise(X[index[cl]], d), d))

# on calcule les logs proba pour tous les modèles et pour tous les signaux de l'échantillon de tests
proba = np.array([[probaSequence(Xd_test[i], models[cl][0], models[cl][1]) for i in range(len(Xd_test))]
                  for cl in range(len(np.unique(Y)))])

# calcul d'une version numérique des Y :
Ynum_test = np.zeros(Y[it].shape)
for num, char in enumerate(np.unique(Y)):
    Ynum_test[Y[it] == char] = num

# pour être sûr, on affiche étiquettes symboliques, celles numériques et une prédiction
print(Y[it])
print(np.array(Ynum_test, dtype=int))
    
# Calcul de la classe la plus probable :
pred = proba.argmax(0) # max colonne par colonne
print(pred)

# Calcul d'un pourcentage de bonne classification :
np.where(pred != Ynum_test, 0.,1.).mean()

# On peut conclure que notre modèle reconnaît plus ou moins des lettres introduites. En fait,
# pour tout d compris dans [3..10] on obtient en moyen plus de moitié de formes bien reconnues
# parmis l'échantillon de tests. De plus, on voit qu'on atteint la meilleur classification
# en moyen lorsque d = 5, avec un pourcentage moyen de bonne classification de ≈68.333%.
# Néanmoins, il semble intéressant de tester ce modèle sur les données différentes,
# particulièrement, sur une autre échantillon de lettres. En outre, il faut aussi appliquer
# notre modèle à l'échantillon mélangé pour considérer les écriture diverses.


['a' 'a' 'a' 'b' 'b' 'b' 'c' 'c' 'c' 'd' 'd' 'd' 'e' 'e' 'e' 'f' 'f' 'f'
 'g' 'g' 'g' 'h' 'h' 'h' 'i' 'i' 'j' 'j' 'k' 'k' 'l' 'l' 'm' 'm' 'n' 'n'
 'o' 'o' 'p' 'p' 'q' 'q' 'r' 'r' 's' 's' 't' 't' 'u' 'u' 'v' 'v' 'w' 'w'
 'x' 'x' 'y' 'y' 'z' 'z']
[ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4  5  5  5  6  6  6  7  7  7
  8  8  9  9 10 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19
 20 20 21 21 22 22 23 23 24 24 25 25]
[ 0  0  0  1  1  1  2  2 14  3  0  0  4  4  4  6  5 19  6 19  6  7  7  1
  8 13  9  9 10 10 19 19 12  7 13 13 14 14  1 15 16 16  7 21 18 18 19 19
 14 22 21 21  0  0 23 23  0 24 25 25]


/Users/ivankachaikin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


0.6833333333333333

## Lutter contre le sur-apprentissage
Cette base de données met en lumière le phénomène de sur-apprentissage : il y a peu de données et dès que le nombre d'états augmente, il y a trop peu d'exemple pour estimer correctement les matrices {$A, \pi$}. De nombreuses cases sont donc à 0, voire des lignes entières (d'où la sécurisation du code pour la normalisation des matrices stochastiques).

Ces 0 sont particulièrement discriminants: considérant la classe {$c$}, ils permettent d'éliminer de cette classe tout signal présentant cette caractéristique. Cette règle est trop forte compte tenu de la taille de la base d'apprentissage. Nous proposons une astuce pour palier cette faiblesse : lors du comptage, initialiser les matrices {$A, \pi$} avec ones au lieu de zeros . On fait semblant d'avoir observer une transition de chaque type avant même le début du comptage.

Comparer les performances en test.

In [135]:
# On définit une fonction alternative
def learnMarkovModel_alternative(Xc, d):
    Pi, A = np.ones(d), np.ones((d, d))  #l'initialisation différent
    for Xc_i in Xc:
        Pi[Xc_i[0]] += 1
        for state_index in range(len(Xc_i)-1):
            A[Xc_i[state_index], Xc_i[state_index+1]] += 1
    
    Pi = Pi / Pi.sum() if Pi.sum() != 0 else Pi
    A = A / np.maximum(A.sum(1).reshape(d, 1), 1)
    
    return Pi, A

d = 10                              # paramètre de discrétisation
Xd_train = discretise(X[ia], d)    # application de la discrétisation
Xd_test = discretise(X[it], d)
index = itrain.copy()
models = []
for cl in range(len(np.unique(Y))): # parcours de toutes les classes et optimisation des modèles
    models.append(learnMarkovModel_alternative(discretise(X[index[cl]], d), d))


proba_alt = np.array([[probaSequence(Xd_test[i], models[cl][0], models[cl][1]) for i in range(len(Xd_test))]
                      for cl in range(len(np.unique(Y)))])

# calcul d'une version numérique des Y :
Ynum_test_alt = np.zeros(Y[it].shape)
for num, char in enumerate(np.unique(Y)):
    Ynum_test_alt[Y[it] == char] = num
    
# Calcul de la classe la plus probable :
pred_alt = proba_alt.argmax(0) # max colonne par colonne

# Calcul d'un pourcentage de bonne classification :
np.where(pred_alt != Ynum_test_alt, 0.,1.).mean()

# Dans ce cas, on peut observer l'amélioration des résultats. Par contre, il ne s'agit pas de
# changements significatifs. En général, on voit qu'on améliore plutôt la précision de
# classification pour plusieurs d tandis que celle maximale de type maximale reste assez proche.
# Par exemple, on obtient souvent ici l'exactitude maximale de 70% au lieu de ≈68.333% du
# modèle précédent

0.7

# Partie optionnelle
## Evaluation qualitative

Nous nous demandons maintenant où se trouvent les erreurs que nous avons commises...

Calcul de la matrice de confusion: pour chaque échantillon de test, nous avons une prédiction (issue du modèle) et une vérité terrain (la vraie étiquette). En posant Nc le nombre de classes, la matrice de confusion est une matrice (Nc x Nc) où nous comptons le nombre d'échantillon de test dans chaque catégorie :

- Initialisation à 0 : 

In [136]:
conf = np.zeros((26,26), dtype=int)

- Pour chaque échantillon, incrément de la case (prediction, vérité)

In [137]:
# Parcours de toutes étiquettes et de toutes predictions
for cl_pred, cl_real in zip(pred, np.array(Ynum_test, dtype=int)):
    conf[cl_pred, cl_real] += 1

- Tracé de la matrice : 

In [138]:
plt.figure()
plt.imshow(conf, interpolation = 'nearest')
plt.colorbar()
plt.xticks(np.arange(26), np.unique(Y))
plt.yticks(np.arange(26), np.unique(Y))
plt.xlabel(u'Vérité terrain')
plt.ylabel(u'Prédiction')
plt.savefig("mat_conf_lettres.png")
# De l'image obtenu on peut déduire que notre modèle reconnaît mal des lettres qui sont pareil
# dans l'écriture française. Par exemple, on s'embrouille souvent dans les lettre 'a' et 'd',
# 'g' et 'q', 't' et 'f', 'o' et 'b', 'a' et 'p'.

## Modèle génératif

Utiliser les modèles appris pour générer de nouvelles lettres manuscrites.

### Tirage selon une loi de probabilité discrète

- faire la somme cumulée de la loi {$sc$}
- tirer un nombre aléatoire {$t$} entre 0 et 1
- trouver la première valeur de {$sc$} qui est supérieure à {$t$}
- retourner cet état 

**Note :** comme vu en cours, tout repose sur la somme cumulée (notée ici `sc$`, calculable en appelant `np.cumsum`. Sur un exemple: la loi `V = [0.2, 0.4, 0.3, 0.1]` a pour somme cumulée `V.cumsum() == [0.2,  0.6,  0.9,  1.0]`

### Génération d'une séquence de longueur N

- tirer un état {$s_0$} selon Pi
- tant que la longueur n'est pas atteinte :
  - tirer un état {$s_{t+1}$} selon {$A[s_{t}]$} 

In [140]:
def generate(Pi, A, d):
    Pi_temp = Pi.copy()
    res = []
    
    # pour chaque état qu'il faut remplir
    for _ in range(states_num):
        # on génére une valeur aléatoire de la distribution uniforme dans [0, 1]
        rand_val = np.random.rand()
#         print(rand_val)
#         print(Pi_temp)
#         print(Pi_temp.cumsum())
        # on prend un état avec sa probabilité
        state = np.arange(d)[Pi_temp.cumsum() > rand_val][0]
#         print(state)
#         print()
        # on stocke un état courant
        res.append(state)
        # on met-à-jour les probabilités
        Pi_temp = np.dot(Pi_temp, A)
        
    return np.array(res)
    
newa = generate(models[0][0], models[0][1], d)

### Affichage du résultat

In [70]:
newa = generate(models[0][0], models[0][1], d)       # generation d'une séquence d'états
intervalle = 360. / d                                 # pour passer des états => valeur d'angles
newa_continu = np.array([i * intervalle for i in newa]) # conv int => double
tracerLettre(newa_continu)
# On voit donc que ce modèle ne fonctionne pas vraiment bien pour génération des lettres